# UPennContrast Sandbox

1. Set up sandbox by installing Girder and Annotation Client.

In [ ]:
!git clone https://github.com/Kitware/UPennContrast/
%cd UPennContrast
!git checkout add-annotation-worker
%pip install -r devops/girder/annotation_client/requirements.txt
%pip install -e devops/girder/annotation_client/
%pip install pandas
%pip install imageio

import IPython
IPython.Application.instance().kernel.do_shutdown(True)

2. Import all necessary packages.

In [ ]:
import girder_client
import annotation_client.annotations as annotations
import annotation_client.tiles as tiles
import pandas as pd
import imageio
pd.set_option('display.max_colwidth', None)

2. Specify apiUrl and account information for authentication.

In [ ]:
apiUrl = 'http://localhost:8080/api/v1'
username = 'admin'
password = 'password'

3. Create an instance of the Annotation Client.

In [ ]:
client = girder_client.GirderClient(apiUrl=apiUrl)
user_id = client.authenticate(username=username, password=password)['_id']
annotationClient = annotations.UPennContrastAnnotationClient(apiUrl=apiUrl, token=client.token)

4. Obtain and list all datasets.

In [ ]:
datasets = dict()
for folder in annotationClient.client.listFolder(user_id,'user'):
    for dataset in annotationClient.client.listFolder(folder['_id']):
        if dataset['meta']['subtype'] == 'contrastDataset':
            datasets[dataset['name']] = dataset['_id']

pd.DataFrame(datasets.keys(),columns=['Dataset Name'])

5. Select the name of a dataset.

In [ ]:
dataset_name = ''

6. Create an instance of the dataset client.

In [ ]:
datasetId = datasets[dataset_name]
datasetClient = tiles.UPennContrastDataset(apiUrl=apiUrl, token=client.token, datasetId=datasetId)

7. Write and test out your algorithm. Below are some examples of common data structures to request from the API.

In [ ]:
# Obtain a list of point annotations

annotationList = annotationClient.getAnnotationsByDatasetId(datasetId, shape='point')

In [ ]:
# Obtain a list of blob annotations

annotationList = annotationClient.getAnnotationsByDatasetId(datasetId, shape='polygon')

In [ ]:
# Obtain a list of connections

connectionList = annotationClient.getAnnotationConnections(datasetId)

In [ ]:
# Obtain the raw image for each annotation

for annotation in annotationList:

    channel = annotation['channel']
    location = annotation['location']
    time, z, xy = location['Time'], location['Z'], location['XY']
    frame = datasetClient.coordinatesToFrameIndex(xy, z, time, channel)
    image = datasetClient.getRegion(datasetId, frame=frame)